In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn.pipeline import Pipeline
from diagnostics.evaluation import fit_and_eval, rev_weighted_quad_kappa
from transformers.one_hot_encoder import CustomOneHotEncoder
from tools.datasets import get_boston, get_heart, get_houses, get_prudential
from sklearn.metrics import make_scorer, accuracy_score, f1_score, mean_squared_error
from imputers.zero_filler import ZeroFiller
from transformers import LabelsClipper
import warnings
warnings.filterwarnings('ignore')

In [2]:

# data, labels, continuous, discrete, dummy, categorical, target = get_boston()
# data, labels, continuous, discrete, dummy, categorical, target = get_heart()
# data, labels, continuous, discrete, dummy, categorical, target = get_houses()
data, labels, continuous, discrete, dummy, categorical, target = get_prudential()
train = data.drop(target, axis=1)

In [4]:
model = Pipeline([
        ('onehot',       CustomOneHotEncoder(columns=categorical)),
        ('zero_filler',  ZeroFiller()),
        # ('classifier',   LogisticRegression()),                # for Heart Disease
        ('classifier',   LabelsClipper(LinearRegression())), # for Prudential
        # ('regressor',    LinearRegression()),                # for Boston and Houses
])

In [9]:
# scores = cross_val_score(model, train.copy(), labels, cv=3, n_jobs=-1, scoring=make_scorer(accuracy_score))
# scores = cross_val_score(model, train.copy(), labels, cv=KFold(3, shuffle=True, random_state=0), 
#                          n_jobs=-1, scoring=make_scorer(mean_squared_error))
scores = cross_val_score(model, train.copy(), labels, cv=3, n_jobs=-1, scoring=make_scorer(rev_weighted_quad_kappa))
# scores = np.sqrt(scores)
print('mean:', scores.mean(),'\nstd:', scores.std())

mean: 0.5375788752266611 
std: 0.008538734001619235
